In [28]:
include("../lib/Lib.jl")
include("../lib/DeepSpace.jl")

using DataFrames, DataFramesMeta
using CSV
using Statistics
using ScikitLearn
using Main

In [29]:
@sk_import linear_model: LinearRegression
@sk_import linear_model: ElasticNet

PyObject <class 'sklearn.linear_model.coordinate_descent.ElasticNet'>

In [30]:
KEY = "2019vahay_sat"
YEAR = "2019"
COLUMN = :totalPoints

:totalPoints

In [31]:
(headers, scores, teams) = Lib.read_stacked(KEY, YEAR)

(["adjustPoints", "autoPoints", "bay1", "bay2", "bay3", "bay4", "bay5", "bay6", "bay7", "bay8"  …  "techFoulCount", "teleopPoints", "topLeftRocketFar", "topLeftRocketNear", "topRightRocketFar", "topRightRocketNear", "totalPoints", "key", "level", "event"], 108×54 DataFrame. Omitted printing of 50 columns
│ Row │ adjustPoints │ autoPoints │ bay1          │ bay2          │
│     │ Int64⍰       │ Int64⍰     │ String⍰       │ String⍰       │
├─────┼──────────────┼────────────┼───────────────┼───────────────┤
│ 1   │ 0            │ 9          │ Panel         │ Panel         │
│ 2   │ 0            │ 12         │ Panel         │ Panel         │
│ 3   │ 0            │ 6          │ None          │ Panel         │
│ 4   │ 0            │ 15         │ Panel         │ PanelAndCargo │
│ 5   │ 0            │ 3          │ None          │ None          │
│ 6   │ 0            │ 12         │ None          │ None          │
│ 7   │ 0            │ 6          │ None          │ PanelAndCargo │
│ 8   │ 0     

In [32]:
scores_train = @linq scores |> where(:level .== "qm")
teams_train = @linq teams |> where(:level .== "qm")

scores_test = @linq scores |> where(:level .!= "qm")
teams_test = @linq teams |> where(:level .!= "qm")

(ß, teamnums, teamindex) = Lib.opr_ß(scores_train, teams_train, shortevent=true)
y = convert.(Float64, coalesce(scores_train[COLUMN], -1.0))

108-element Array{Float64,1}:
 32.0
 27.0
 23.0
 50.0
 26.0
 46.0
 38.0
 23.0
 25.0
 33.0
 51.0
 18.0
 48.0
  ⋮  
 26.0
 92.0
 65.0
 37.0
 19.0
 19.0
 38.0
 28.0
 39.0
  2.0
 69.0
 53.0

In [33]:
model1 = ScikitLearn.fit!(LinearRegression(fit_intercept=false), ß, y)

PyObject LinearRegression(copy_X=True, fit_intercept=False, n_jobs=None,
         normalize=False)

In [34]:
oprs = DataFrame(hcat(teamnums, model1[:coef_]), [:team, :rating])
#sort(oprs, (:rating), rev = true) |> print

,team,rating
,Any,Any
1,frc1123,5.66419
2,frc116,14.0356
3,frc1418,20.8558
4,frc1629,13.8876
5,frc1719,18.4075
6,frc1731,6.10189
7,frc1895,20.9665
8,frc2028,11.5811
9,frc2186,6.9579


In [35]:
model2 = ScikitLearn.fit!(ElasticNet(alpha=0, l1_ratio=0, fit_intercept=true), ß, y)
oprs2 = DataFrame(hcat(teamnums, model2[:coef_]), [:team, :rating])
sort(oprs2, (:rating), rev = true) |> print

38×2 DataFrame
│ Row │ team    │ rating    │
│     │ Any     │ Any       │
├─────┼─────────┼───────────┤
│ 1   │ frc614  │ 20.1808   │
│ 2   │ frc2534 │ 11.6696   │
│ 3   │ frc612  │ 9.16499   │
│ 4   │ frc2363 │ 8.96938   │
│ 5   │ frc1895 │ 8.20119   │
│ 6   │ frc1418 │ 8.09044   │
│ 7   │ frc5338 │ 7.44064   │
│ 8   │ frc316  │ 7.2233    │
│ 9   │ frc4242 │ 6.4269    │
│ 10  │ frc1719 │ 5.64216   │
│ 11  │ frc2988 │ 5.47273   │
│ 12  │ frc888  │ 3.84417   │
│ 13  │ frc611  │ 3.47854   │
│ 14  │ frc2199 │ 3.38525   │
│ 15  │ frc4505 │ 3.13804   │
│ 16  │ frc3455 │ 2.78969   │
│ 17  │ frc4099 │ 2.30694   │
│ 18  │ frc116  │ 1.27028   │
│ 19  │ frc1629 │ 1.12224   │
│ 20  │ frc3373 │ 0.757425  │
│ 21  │ frc3941 │ 0.702704  │
│ 22  │ frc6504 │ -0.871413 │
│ 23  │ frc5549 │ -1.05947  │
│ 24  │ frc2028 │ -1.18422  │
│ 25  │ frc339  │ -1.71865  │
│ 26  │ frc6543 │ -2.29536  │
│ 27  │ frc6584 │ -4.29622  │
│ 28  │ frc623  │ -4.45212  │
│ 29  │ frc2186 │ -5.80744  │
│ 30  │ frc1731 │ -6.6634

In [36]:
(ßtest, _, _) = Lib.opr_ß(scores_test, teams_test, teamnums=teamnums)

(Array{Float64}(0,38), Union{Missing, String}["frc1123", "frc116", "frc1418", "frc1629", "frc1719", "frc1731", "frc1895", "frc2028", "frc2186", "frc2199"  …  "frc611", "frc612", "frc614", "frc620", "frc623", "frc6504", "frc6543", "frc6584", "frc6863", "frc888"], Dict("frc888"=>38,"frc4505"=>26,"frc2421"=>12,"frc2819"=>14,"frc3941"=>22,"frc2961"=>15,"frc3274"=>18,"frc620"=>32,"frc2199"=>10,"frc4242"=>24…))

In [37]:
test_y = convert.(Float64, coalesce(scores_test[COLUMN], -1.0))
test_yhat_m1 = ScikitLearn.predict(model1, ßtest)
test_yhat_m2 = ScikitLearn.predict(model2, ßtest)

PyCall.PyError: PyError ($(Expr(:escape, :(ccall(#= /Users/wes/.julia/packages/PyCall/0jMpb/src/pyfncall.jl:44 =# @pysym(:PyObject_Call), PyPtr, (PyPtr, PyPtr, PyPtr), o, pyargsptr, kw))))) <class 'ValueError'>
ValueError('Found array with 0 sample(s) (shape=(0, 38)) while a minimum of 1 is required.')
  File "/Users/wes/.julia/conda/3/lib/python3.7/site-packages/sklearn/linear_model/base.py", line 213, in predict
    return self._decision_function(X)
  File "/Users/wes/.julia/conda/3/lib/python3.7/site-packages/sklearn/linear_model/base.py", line 196, in _decision_function
    X = check_array(X, accept_sparse=['csr', 'csc', 'coo'])
  File "/Users/wes/.julia/conda/3/lib/python3.7/site-packages/sklearn/utils/validation.py", line 582, in check_array
    context))


In [38]:
predictions_table = DataFrame(hcat(scores_test.key, test_y, test_yhat_m1, test_yhat_m2), [:key, :y, :LinReg, :RidgeReg])

UndefVarError: UndefVarError: test_yhat_m1 not defined

In [39]:
rmse_1 = mean((predictions_table.LinReg-predictions_table.y).^2) |> sqrt

UndefVarError: UndefVarError: predictions_table not defined

In [40]:
rmse_2 = mean((predictions_table.RidgeReg-predictions_table.y).^2) |> sqrt

UndefVarError: UndefVarError: predictions_table not defined